In [1]:
import os
import ee
import requests
import numpy as np
from PIL import Image
from io import BytesIO
import geopandas as gpd


In [ ]:
ee.Authenticate()
ee.Initialize()

In [ ]:
# Path to the downloaded and unzipped shapefile
shapefile_path = 'cbp-datasets/cb_2016_us_county_500k/cb_2016_us_county_500k.shp'

# Load the shapefile into a GeoDataFrame
counties = gpd.read_file(shapefile_path)

In [ ]:
# Function to fetch satellite image for each county with reduced resolution
def fetch_image_for_county(geometry, start_date='2016-01-01', end_date='2016-12-31', width=1024, height=1024):
    # Sentinel-2 imagery (can also use Landsat if desired)
    image_collection = ee.ImageCollection("COPERNICUS/S2") \
                        .filterBounds(geometry) \
                        .filterDate(start_date, end_date) \
                        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))  \
                        .sort('system:time_start', False)  # Sort by cloud coverage in descending order
    
    
    # Filter the image collection to exclude images with more than 10% NoData pixels
    #image_collection = image_collection.filter(ee.Filter.lte('nodata_percentage', 10))  # Exclude images with >10% NoData pixels
    
    # Sort by cloud coverage percentage, in ascending order (less cloud)
    image_collection = image_collection.sort('CLOUDY_PIXEL_PERCENTAGE')

    if image_collection.size().getInfo() == 0: 
        return None

    # Get the first (most recent)
    #image = image_collection.first() 
    image_collection = image_collection.limit(100)

    #print(image.propertyNames().getInfo())

    image_list = image_collection.toList(image_collection.size())
    
    for i in range(image_list.size().getInfo()):
        image = ee.Image(image_list.get(i))

        # Select RGB bands (Red, Green, Blue)
        image = image.select(['B4', 'B3', 'B2'])  # Red, Green, Blue bands

        # Visualization parameters for the RGB image
        vis_params = {
            'min': 0,
            'max': 8000,  # Adjust max value for better visualization
            'gamma': 1.4
        }

        # Generate a thumbnail image URL with reduced size
        url = image.getThumbURL({
            'min': 0,
            'max': 8000,
            'gamma': 1.4,
            'dimensions': '1024x1024',  # Resize the image to the specified dimensions
            'resample': 'bilinear',
        })

        response = requests.get(url)

        if response.status_code == 200:
            # Open the image using PIL
            img = Image.open(BytesIO(response.content))
        
            # Check if the image has an alpha channel (RGBA)
            if img.mode == 'RGBA':
                # Convert the image to RGB by discarding the alpha channel
                img = img.convert('RGB')

            # Convert the image to a numpy array
            img_array = np.array(img)

            # Check if the image is black (RGB values are (0, 0, 0))
            black_pixels = np.sum(np.all(img_array == [0, 0, 0], axis=-1))

            # Total number of pixels in the image
            total_pixels = img_array.shape[0] * img_array.shape[1]

            # Calculate the percentage of black pixels
            black_percentage = (black_pixels / total_pixels) * 100 

            if black_percentage <= 10:
                return img     
        
    return None

In [ ]:
folder_path = r"2016-US-Counties-Images-500k"

# Loop through all counties and fetch/save images
for idx, county in counties.iterrows():
    # Get the county geometry (polygon)
    county_geometry = ee.Geometry.Polygon(county['geometry'].__geo_interface__['coordinates'][0])
    
    # Fetch the satellite image URL for the county
    image = fetch_image_for_county(county_geometry, width=1024, height=1024)

    # Get the county name (remove spaces and special characters for a valid filename)
    county_name = county['NAME'].replace(" ", "_").replace(",", "").replace(":", "").replace("/", "_")
    
    if image is None:
        print(f"{county_name}_{county['GEOID']} not saved")
        continue
    
    # Define the path to save the image
    save_path = os.path.join(folder_path, f"{county_name}_{county['GEOID']}.jpg")
            
    # Save the image as JPEG
    image.save(save_path, 'JPEG')
    print(f"Image saved to {save_path}")